导致gurobi报出infeasible model错误的原因有两种:

1. 模型约束条件里面有互相矛盾的地方，这个模型本身并没有可行解。
2. 模型本身没有问题，把数学模型输入到gurobi的过程中，编程错误或者参数输入不正确导致模型不可行。

In [3]:
import gurobi as gp
m = gp.Model("test")
x = m.addVars(2, lb = 0.0, ub = 1.0, name = 'x')
m.addConstr(x[0] + x[1] >= 1.2, 'c1')
m.addConstr(x[0] + x[1] <= 1.1, 'c2')
m.addConstr(x[0] + x[1] >= 0.1, 'c3')

m.setObjective(x[0] + 2 * x[1])
m.optimize()

m.computeIIS()
m.write('model1.ilp')

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x737daf1e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.300000e+00   0.000000e+00      0s

IIS computed: 2 constraints and 0 bounds
IIS runtime: 0.00 seconds


## 处理冲突约束的思路

1. 从上面的代码容易看到第一条约束x0+x1>=1.2和第二条约束x0+x1<=1.1是互相矛盾的，因此该模型不存在可行解。

2. 所以当我们求解这个模型的时候，gurobi会报出 infeasible model 的错误信息，此时我们可以调用computeIIS()即可得到哪些约束是互相矛盾的，即去掉这些矛盾约束剩下的约束构成的问题是可行的。

3. 为了方便观察computellS()的输出结果，可以用m.write('model1.ilp')输出一个扩展名为ilp的文件，里边会包含所有矛盾约束。

```bash
Minimize
 
Subject To
 c1: x[0] + x[1] >= 1.2
 c2: x[0] + x[1] <= 1.1
Bounds
 x[0] free
 x[1] free
End
```

可以看到其输出了前两个造成矛盾的约束，而第三个约束没有矛盾，所以没有输出在`.ilp`文件内。

但是大规模问题用不了`computeIIS()`，会非常慢。